This first cell makes sure that torchgan is installed. It's primarily here for use with colab, since colab doesn't have torchgan installed by default.

In [1]:
# from torchGAN tutorial

try:
    import torchgan

    print(f"Existing TorchGAN {torchgan.__version__} installation found")
except ImportError:
    import subprocess
    import sys

    subprocess.check_call([sys.executable, "-m", "pip", "install", "torchgan"])
    import torchgan

    print(f"Installed TorchGAN {torchgan.__version__}")

Existing TorchGAN v0.1.0 installation found


In [1]:
import torch
import torch.nn as nn
import torch.backends.cudnn
import torch.utils.data as tdata
from torch.optim import Adam
import torchvision
import torchvision.datasets as dsets
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

import torchgan
from torchgan.models import *
from torchgan.losses import *
from torchgan.trainer import Trainer

This cell takes care of creating the dataset. Any PNG or JPEG image files are acceptable, as long as their width and height are the same. The models also require that the dimensions of the image are a perfect power of 2, but that can be overcome by scaling the images.

Another thing to note is that the file path provided as root cannot be the exact folder that holds the images. In order for the path to be recognized properly, you must specify the folder that contains the folder with the images.

For copyright reasons, the images that I am training from are not included with the GitHub repo. If you are interested in learning more, the citations for all of the images are included in the bibliography of the accompanying report.

In [39]:
size = 64
channels = 3 # change to 1 if using grayscale
data = ImageFolder(
    root="./data/pictures/wikiart/monet/trainA/",
    transform=transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor(),
        #transforms.Grayscale(),
        transforms.Normalize(mean=.5, std=.5)
    ])
)
print(len(data))

1072


In [40]:
dataloader = tdata.DataLoader(data, batch_size=64, shuffle=True)

In [5]:
# adapted from torchgan tutorial

dcgan_network = {
    "generator": {
        "name": DCGANGenerator,
        "args": {
            "encoding_dims": 100,
            "out_size": size,           # this is how many pixels wide and tall the generated images will be
            "out_channels": channels,
            "step_channels": 32,
            "nonlinearity": nn.LeakyReLU(0.2),
            "last_nonlinearity": nn.Tanh(),
        },
        "optimizer": {"name": Adam, "args": {"lr": 0.0001, "betas": (0.5, 0.999)}},
    },
    "discriminator": {
        "name": DCGANDiscriminator,
        "args": {
            "in_size": size,            # this is how many pixels wide and tall we resized the images to be.
            "in_channels": channels,
            "step_channels": 32,
            "nonlinearity": nn.LeakyReLU(0.2),
            "last_nonlinearity": nn.LeakyReLU(0.2),
        },
        "optimizer": {"name": Adam, "args": {"lr": 0.0003, "betas": (0.5, 0.999)}},
    },
}

In [42]:
# from torchgan tutorial

minimax_losses = [MinimaxGeneratorLoss(), MinimaxDiscriminatorLoss()]
wgangp_losses = [
    WassersteinGeneratorLoss(),
    WassersteinDiscriminatorLoss(),
    WassersteinGradientPenalty(),
]
lsgan_losses = [LeastSquaresGeneratorLoss(), LeastSquaresDiscriminatorLoss()]

If you're running this on colab, be sure to select a GPU runtime for the best performance.

In [43]:
# more from torchgan tutorial

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    # Use deterministic cudnn algorithms
    torch.backends.cudnn.deterministic = True
    epochs = 10
else:
    device = torch.device("cpu")
    epochs = 5

print("Device: {}".format(device))
print("Epochs: {}".format(epochs))

Device: cpu
Epochs: 5


In [44]:
# from torchgan tutorial

trainer = Trainer(
    dcgan_network,
    lsgan_losses,
    sample_size=64,
    epochs=500,             # you can override the number of epochs here if you want to try to get better results
    device=device
)

This is the cell that actually does the training. After it finishes, use the next cell if you want to keep training the same model.

In [45]:
trainer(dataloader)

Saving Model at './model/gan0.model'
Epoch 1 Summary
Epoch time duration : 166.5189619064331
generator Mean Gradients : 1431.7426402364401
discriminator Mean Gradients : 2242.0133246193677
Mean Running Discriminator Loss : 2.0294051240472233
Mean Running Generator Loss : 1.8131286852500017
Generating and Saving Images to ./images/epoch1_generator.png

Saving Model at './model/gan1.model'
Epoch 2 Summary
Epoch time duration : 154.58307313919067
generator Mean Gradients : 932.107507486842
discriminator Mean Gradients : 1761.3530588865247
Mean Running Discriminator Loss : 1.8800436135600596
Mean Running Generator Loss : 1.5885166736210095
Generating and Saving Images to ./images/epoch2_generator.png

Saving Model at './model/gan2.model'
Epoch 3 Summary
Epoch time duration : 149.38809490203857
generator Mean Gradients : 726.9682092436076
discriminator Mean Gradients : 1357.4498150129837
Mean Running Discriminator Loss : 1.5449869924900579
Mean Running Generator Loss : 1.455603397360035
Gen

Use this cell instead of the previous one if you want to resume training a model that you saved from a previous session. Be sure to use the correct file path and file name when you call the function.

In [38]:
trainer.load_model("./model/epoch4000.model")
trainer(dataloader)

Loading Model From './model/epoch4000.model' to cpu
Saving Model at './model/gan0.model'
Epoch 4001 Summary
Epoch time duration : 61.66512179374695
generator Mean Gradients : 0.1942862312819429
discriminator Mean Gradients : 0.41133713040341746
Mean Running Discriminator Loss : 0.047958257390961836
Mean Running Generator Loss : 0.5399657180436379
Generating and Saving Images to ./images/epoch1_generator.png

Saving Model at './model/gan1.model'
Epoch 4002 Summary
Epoch time duration : 57.954326152801514
generator Mean Gradients : 0.1825847586664016
discriminator Mean Gradients : 0.27678671105862807
Mean Running Discriminator Loss : 0.047946959392419476
Mean Running Generator Loss : 0.5399632810736945
Generating and Saving Images to ./images/epoch2_generator.png

Saving Model at './model/gan2.model'
Epoch 4003 Summary
Epoch time duration : 58.11652660369873
generator Mean Gradients : 0.25772636286740697
discriminator Mean Gradients : 0.4817640196516286
Mean Running Discriminator Loss : 